<a href="https://colab.research.google.com/github/bxck75/audio_video_swapper/blob/master/audio_video_swapperV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A/V Swapper
## the automated process does the following
*   Rip the videos
*   Split audio from video
*   Do some corrections in tempo of audio/video
*   Swap the audio of two youtube video's
*   Stick audio of B(Audio video) under A(Dancer video)
*   Do manual corrections after the main process






In [0]:
# { output-height: 1, display-mode: "form" }
!rm -r /content/sample_data
#@title Setup the swap 
import subprocess,os,sys 
from IPython.display import clear_output
# install youtube-dl
!pip install youtube-dl ffmpeg moviepy imageio
import moviepy,os
import moviepy.editor as mp
import imageio
from IPython.display import clear_output
from moviepy.editor import VideoFileClip, AudioFileClip,concatenate_audioclips,concatenate_videoclips,clips_array, vfx,CompositeVideoClip
from moviepy.video.fx import crop
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import zipfile

#functions
def system_call(command):
    p = subprocess.Popen([command], stdout=subprocess.PIPE, shell=True)
    return p.stdout.read()

def get_seconds(timestr):
  hms = [60,1,0]
  return str(sum([a*b for a,b in zip(hms, map(int,timestr.split(':')))]))

def Trim_video(t,tr,start=0,stop=1000):  
  # cut clip if need
#   !ffmpeg -hwaccel cuvid -y -i {t} -ss {start} -async 1 -strict -2 {tr}
#   ffmpeg_extract_subclip(t, start, stop, targetname=tr) 
  with VideoFileClip(t) as video:
      new = video.subclip(start, stop)
      new.write_videofile(tr, audio_codec='aac')

# ipywidgets git cloene
system_call('git clone https://github.com/jupyter-widgets/ipywidgets.git /content/audioswapper/ipywidgets')
# ipywidgets install
system_call('pip install ipywidgets && jupyter nbextension enable --py --sys-prefix widgetsnbextension')


# some vids to test
# https://www.youtube.com/watch?v=LeKxHwB1oVA
# https://www.youtube.com/watch?v=vdJcJAO8lrE
# https://www.youtube.com/watch?v=5ww-pQynEDI # audio
# https://www.youtube.com/watch?v=19WUwZYM7bM # audio
# https://www.youtube.com/watch?v=FHWQHeThuz0 # techno

#Make folder for extraction
if not os.path.exists('/content/audioswapper'):
  os.mkdir('/content/audioswapper',777)
if not os.path.exists('/content/final_result'):
  os.mkdir('/content/final_result',777)

#@markdown >video to get the dancer video from
DanceVideo='https://www.youtube.com/watch?v=Ee2hKlGXrDs' #@param {type:"string"}
#@markdown >video to get the audio from
AudioVideo='https://www.youtube.com/watch?v=SYnVYJDxu2Q' #@param {type:"string"}
#@markdown >speed adjust for the video source
DanceSpeed = 1 #@param {type:"slider", min:0.01, max:2.01, step:0.01}
#@markdown  >speed adjust for the audio source
AudioSpeed = 1.1 #@param {type:"slider", min:0.01, max:2.01, step:0.01}
clear_output()

In [0]:
#@title Run this cell to fetch the videos and see the length { output-height: 1, display-mode: "form" }
system_call('youtube-dl -f mp4 '+AudioVideo+' --output /content/audioswapper/audiosource.mp4')
audio_time = system_call("ffmpeg -i /content/audioswapper/audiosource.mp4 2>&1 | grep 'Duration' |awk -F '00:' '{print $2}' |awk -F ',' '{print $1}'")
system_call('youtube-dl -f mp4 '+DanceVideo+' --output /content/audioswapper/videosource.mp4')
video_time = system_call("ffmpeg -i /content/audioswapper/videosource.mp4 2>&1 | grep 'Duration' |awk -F '00:' '{print $2}' |awk -F ',' '{print $1}'")

print(audio_time)
print(video_time)

clat=get_seconds(str(audio_time).split("'")[1].replace(".",":").replace("\\n",""))
clvt=get_seconds(str(video_time).split("'")[1].replace(".",":").replace("\\n",""))
clear_output()
print("Audio source has "+clat+" seconds")
print("Video source has "+clvt+" seconds")

In [0]:
# @title Trim begin and end { output-height: 0, display-mode: "form" }
#@markdown (make time adjustments with sliders)

Trim_Start_Dance= 0 #@param {type: "slider", min: 0, max: 200}
Trim_Start_Audio= 0 #@param {type: "slider", min: 0, max: 200}
Trim_End_Dance= 612 #@param {type: "slider", min: 0, max: 1200}
Trim_End_Audio = 564 #@param {type:"slider", min:0, max:1200, step:1}
Final_output_name = 'output_video' #@param {type: "string"}

#check end times
input('set the start and end of sources and press enter')
l=[int(clat)-Trim_Start_Audio,int(clvt)-Trim_Start_Dance]
shortest_video=min(int(i) for i in l)
print('The shortest video is '+str(shortest_video))

if Trim_End_Dance > int(clvt):
  Trim_End_Dance = int(clvt)
  print('set the end time of Videosource to the file limit '+str(shortest_video))
if Trim_End_Audio > int(shortest_video):
  print('set the end time of Audiosource to the file limit '+str(shortest_video))
  Trim_End_Audio = int(shortest_video)
clear_output()

# Automated process

In [0]:
# trim the audio  
print(str(Trim_Start_Audio)+'/'+str(Trim_End_Audio))
Trim_video('/content/audioswapper/audiosource.mp4','/content/audioswapper/VI_trimmed.mp4',Trim_Start_Audio,Trim_End_Audio)
# trim the video
print(str(Trim_Start_Dance)+'/'+str(Trim_End_Dance))
Trim_video('/content/audioswapper/videosource.mp4','/content/audioswapper/AU_trimmed.mp4',Trim_Start_Dance,Trim_End_Dance)

# speed up dance vid
print('working on speeding')
system_call('ffmpeg -y -hwaccel cuvid -i "/content/audioswapper/AU_trimmed.mp4" \
  -filter_complex "[0:v]setpts='+str(DanceSpeed)+'*PTS[v];[0:a]atempo=1.0[a]" -map "[v]" \
  -map "[a]" "/content/audioswapper/AU_up_speed.mp4"')
system_call('ffmpeg -y -hwaccel cuvid -i "/content/audioswapper/VI_trimmed.mp4" \
  -filter_complex "[0:v]setpts=0.5*PTS[v];[0:a]atempo='+str(AudioSpeed)+'[a]" -map "[v]" \
  -map "[a]" "/content/audioswapper/VI_up_speed.mp4"')

# split the audio from de trimmed video
videoclip = VideoFileClip('/content/audioswapper/VI_up_speed.mp4')
audioclip = videoclip.audio
audioclip.write_audiofile('/content/audioswapper/VI.mp3')

# sync audio video
system_call('ffmpeg -y -hwaccel cuvid -i /content/audioswapper/AU_up_speed.mp4 \
  -i /content/audioswapper/VI.mp3 -af "adelay=6500|6500" -c:v copy /content/audioswapper/synced_AU.mp4')

# load the Video source
video = VideoFileClip("/content/audioswapper/synced_AU.mp4")
# combine with the Audio source
video_with_audio = video.set_audio(AudioFileClip("/content/audioswapper/VI.mp3")) 
# write the output video
video_with_audio.write_videofile("/content/final_result/"+Final_output_name+".mp4", fps=60, codec="mpeg4")

clear_output()

#### The End of the automated process

In [0]:
sys.exit('The end of the swapping....manual below')
# system_call('rm -r /content/audioswapper')
clear_output()

# Manual adjustments

In [0]:
#@title Youtube Ripper
#@markdown type the youtube url and filename
url_to_rip = "https://www.youtube.com/watch?v=bwnSkNyWrxI" #@param {type: "string"}
mpg_output_file = "/content/audioswapper/techno_berlin.mp4" #@param {type: "string"}
!youtube-dl -f mp4 --output {mpg_output_file} {url_to_rip}
clear_output()

In [0]:
# system_call('ffmpeg -y -hwaccel cuvid -i "/content/final_result/idiotfinal.mp4" \
#   -filter_complex "[0:v]setpts=1.10*PTS[v];[0:a]atempo=1.41[a]" -map "[v]" \
#   -map "[a]" "/content/final_result/idiotfinal_tempo3.mp4"')
#@title A/V speed adjust
#@markdown Trim audio and video speed with the sliders 1.00 is no change
org_vid= '/content/boogie_AUenshuffle1_russia.mp4' #@param {type : "string"}
new_vid= "/content/russia_sub_clip_speed.mp4" #@param {type : "string"}
audio_speed = 1.02 #@param {type: "slider", min: 0.01, max: 2.01, step:0.01}
video_speed = 0.83 #@param {type:"slider", min:0.0, max:2.01, step:0.01}
args='[0:v]setpts='+str(video_speed)+'*PTS[v];[0:a]atempo='+str(audio_speed)+'[a]" -map "[v]" -map "[a]" ' + new_vid
system_call('ffmpeg -y -hwaccel cuvid -i '+org_vid+' -filter_complex "'+args)
clear_output()

In [0]:
#@title Trimmer
#@markdown Trim the end and begin of a video
org_vid= '/content/russia_sub_clip_speed.mp4' #@param {type : "string"}
new_vid= "/content/russia_sub_clip_speed_cut_final.mp4" #@param {type : "string"}
# /content/boogie_AUenshuffle1_russia_11_00a.mp4
start= '00:00:24' #@param {type : "string"}
end= '00:03:50' #@param {type : "string"}

if new_vid == '':
    name,ext =org_vid.split('.')
    new_vid  = name+'_cut.'+ext
    print(new_vid)


Trim_video(org_vid,new_vid, start,end)

clear_output()

# Experimental and proto code bellow

In [0]:
# reactivate functions
def Trim_video(t,tr,start=0,stop=1000):  
  # cut clip if need
#   !ffmpeg -hwaccel cuvid -y -i {t} -ss {start} -async 1 -strict -2 {tr}
#   ffmpeg_extract_subclip(t, start, stop, targetname=tr) 
  with VideoFileClip(t) as video:
      new = video.subclip(start, stop)
      new.write_videofile(tr, audio_codec='aac')
# Trim_video('/content/final_result/boogie_AUenshuffle1.mp4','/content/final_result/boogie_AUenshuffle_trim.mp4',32,278)
def get_seconds(timestr):
  ftr = [3600,60,1]
  return str(sum([a*b for a,b in zip(ftr, map(int,timestr.split(':')))]))
def system_call(command):
    p = subprocess.Popen([command], stdout=subprocess.PIPE, shell=True)
    return p.stdout.read()
def get_vid_length(V)
    system_call('youtube-dl -f mp4 '+V+' --output /content/audioswapper/audiotemp.mp4')
    return system_call("ffmpeg -i /content/audioswapper/temp.mp4 2>&1 | grep 'Duration' |awk -F '00:' '{print $2}' |awk -F ',' '{print $1}'")



In [0]:
# Trim_video('/content/audioswapper/audiosourceberlin.mp4','/content/audioswapper/audiosourceberlintrimmerd_dance_talk.mp4',30,40)
# Trim_video('/content/audioswapper/audiosourceberlin.mp4','/content/audioswapper/audiosourceberlintrimmerd_dance1.mp4',20,30)
# Trim_video('/content/audioswapper/audiosourceberlin.mp4','/content/audioswapper/audiosourceberlintrimmerd_dance2.mp4',44,68)
# Trim_video('/content/audioswapper/audiosourceberlin.mp4','/content/audioswapper/audiosourceberlintrimmerd_dance3.mp4',71,96)
# Trim_video('/content/audioswapper/audiosourceberlin.mp4','/content/audioswapper/audiosourceberlintrimmerd_dance4.mp4',108,152)
# Trim_video('/content/audioswapper/audiosourceberlin.mp4','/content/audioswapper/audiosourceberlintrimmerd_dance5.mp4',176 ,180)
Trim_video('/content/audioswapper/audiosourceberlin.mp4','/content/audioswapper/audiosourceberlintrimmerd_dance6.mp4',200 ,240)
Trim_video('/content/audioswapper/audiosourceberlin.mp4','/content/audioswapper/audiosourceberlintrimmerd_dance7.mp4',240 ,260)

In [0]:
# /content/audioswapper/VI.mp3 biggie
video = VideoFileClip("/content/audioswapper/VI_trimmed_new.mp4")
audio=video.audio
audio.write_audiofile('/content/audioswapper/VIlol.mp3')

# video = VideoFileClip("/content/final_result/synced_AU.mp4")
# video_with_new_audio = video.set_audio(AudioFileClip("/content/audioswapper/VI.mp3")) 
videoclip1=VideoFileClip('/content/audioswapper/audiosourceberlintrimmerd_dance_talk.mp4').volumex(0.0)
videoclip2=VideoFileClip('/content/audioswapper/audiosourceberlintrimmerd_dance1.mp4').volumex(0.0)
videoclip3=VideoFileClip('/content/audioswapper/audiosourceberlintrimmerd_dance2.mp4').volumex(0.0)
videoclip4=VideoFileClip('/content/audioswapper/audiosourceberlintrimmerd_dance3.mp4').volumex(0.0)
videoclip5=VideoFileClip('/content/audioswapper/audiosourceberlintrimmerd_dance4.mp4').volumex(0.0)
videoclip6=VideoFileClip('/content/audioswapper/audiosourceberlintrimmerd_dance5.mp4').volumex(0.0)
videoclip7=VideoFileClip('/content/audioswapper/audiosourceberlintrimmerd_dance6.mp4').volumex(0.0)
videoclip7=VideoFileClip('/content/audioswapper/audiosourceberlintrimmerd_dance7.mp4').volumex(0.0)

concat_clip = mp.concatenate_videoclips([videoclip1,videoclip2,videoclip3,videoclip4,videoclip5,videoclip6,videoclip7], method="compose")
# concat_clip.write_videofile("texts.mp4")
# final = CompositeVideoClip([videoclip1,videoclip2,videoclip3,videoclip4])
# final.set_audio(AudioFileClip("/content/audioswapper/VIlol.mp3")).volumex(1.4)
concat_clip.write_videofile("/content/compovid2.mp4")#audio="/content/audioswapper/VIlol.mp3"
# sync audio video
system_call('ffmpeg -y -hwaccel cuvid -i /content/compovid2.mp4 -i /content/audioswapper/VIlol.mp3 -af "adelay=6500|6500" -c:v copy /content/final_result/synced_AU.mp4')
video = VideoFileClip("/content/final_result/synced_AU.mp4")
# video = VideoFileClip("/content/final_result/synced_AU.mp4")
video_with_new_audio = video.set_audio(AudioFileClip("/content/audioswapper/VIlol.mp3")) 
# video_with_new_audio.write_videofile("/content/final_result/VI_AU.mp4", fps=60, codec="mpeg4")
video_with_new_audio.write_videofile("/content/final_result/idiotfinal.mp4")


##widgets

In [0]:
get_seconds('00:00:00')

In [0]:
# video_time = system_call("ffmpeg -i /content/final_result/idiotfinal_tempo3.mp4 2>&1 | grep 'Duration' ")#|awk -F '00:' '{print $2}' |awk -F ',' '{print $1}'


print(video_time)
print(get_seconds('00:03:30'))
print(get_vid_length(V))

Trim_video('/content/final_result/idiotfinal_tempo3.mp4','/content/final_result/idiotfinal_tempo3_final.mp4',0,'00:02:22')

In [0]:
# # split the audio from de trimmed video
# system_call('ffmpeg -y -hwaccel cuvid -i "/content/audioswapper/audiosource.mp4" \
#   -filter_complex "[0:v]setpts=0.5*PTS[v];[0:a]atempo=1.21[a]" -map "[v]" \
#   -map "[a]" "/content/audioswapper/VI_up_speed.mp4"')
videoclip = VideoFileClip('/content/audioswapper/audiosource.mp4')
audioclip = videoclip.audio
audioclip.write_audiofile('/content/final_result/VI.mp3')
# sync audio video
system_call('ffmpeg -y -hwaccel cuvid -i /content/compovid.mp4 -i /content/audioswapper/VIlol.mp3 -af "adelay=6500|6500" -c:v copy /content/final_result/synced_AU.mp4')

video = VideoFileClip("/content/final_result/synced_AU.mp4")
# video = VideoFileClip("/content/final_result/synced_AU.mp4")
video_with_new_audio = video.set_audio(AudioFileClip("/content/final_result/VI.mp3")) 
# video_with_new_audio.write_videofile("/content/final_result/VI_AU.mp4", fps=60, codec="mpeg4")
video_with_new_audio.write_videofile("/content/final_result/"+Final_output_name+".mp4")

In [0]:
# split the audio from de trimmed video
system_call('ffmpeg -y -hwaccel cuvid -i '/content/final_result/boogie_AUenshuffle1.mp4' \
  -filter_complex "[0:v]setpts=1.2*PTS[v];[0:a]atempo=0.9[a]" -map "[v]" \
  -map "[a]" "/content/final_result/boogie_AUenshuffle1_speed.mp4"')

In [0]:
# /content/2019-02-16_10h23m05.mp3
with AudioFileClip('/content/2019-02-16_10h23m05.mp3') as audio:
      new = audio.subclip(0, 350)
      new.write_audiofile('subclipaudio.mp3')

In [0]:
# sync audio video
system_call('ffmpeg -y -hwaccel cuvid -i /content/audioswapper/AU_up_speed.mp4 \
  -i /content/subclipaudio.mp3 -af "adelay=6500|6500" -c:v copy /content/final_result/synced_AU.mp4')

In [0]:

system_call('ffmpeg -y -hwaccel cuvid -i "/content/final_result/idiotfinal.mp4" \
  -filter_complex "[0:v]setpts=1.2*PTS[v];[0:a]atempo=1.25[a]" -map "[v]" \
  -map "[a]" "/content/final_result/idiotfinal_tempo.mp4"')

In [0]:


video = VideoFileClip("/content/final_speed.mp4")
# video = VideoFileClip("/content/final_result/synced_AU.mp4")
video_with_new_audio = video.set_audio(AudioFileClip("/content/subclipaudio.mp3")) 
# video_with_new_audio.write_videofile("/content/final_result/VI_AU.mp4", fps=60, codec="mpeg4")
video_with_new_audio.write_videofile("/content/ownaudio.mp4", fps=60, codec="mpeg4")

In [0]:
#widgets

# All input widgets!

#@title Example form fields
#@markdown Forms support many types of fields.

no_type_checking = 'name'  #@param
string_type = 'example'  #@param {type: "string"}
slider_value = 179  #@param {type: "slider", min: 100, max: 200}
number = 102  #@param {type: "number"}
date = '2010-11-05'  #@param {type: "date"}
pick_me = "monday"  #@param ['monday', 'tuesday', 'wednesday', 'thursday']
select_or_input = "apples" #@param ["apples", "bananas", "oranges"] {allow-input: true}
#@markdown ---*italicized text* 

In [0]:
# !zip /content/audioswapper/ipywidgets ipywidgets_zip.zip

import numpy as np
from matplotlib import pyplot as plt

ys = 200 + np.random.randn(100)
x = [x for x in range(len(ys))]

plt.plot(x, ys, '-')
plt.fill_between(x, ys, 195, where=(ys > 195), facecolor='g', alpha=0.6)

plt.title("Fills and Alpha Example")
plt.show()